<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/235_PredRevenue_Gap_Orchestrator_Tier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Revenue Gap Orchestrator - Architecture Plan

## 📊 Data Structure Analysis

### Sales Data (`retail_weekly_sales.csv`)
- **Structure:** 2,400 records (200 customers × 12 weeks)
- **Columns:** 16 columns including:
  - Core: `customer_id`, `week_start_date`, `weekly_spend`, `store_id`
  - Temporal: `week_number`, `month`, `month_name`, `quarter`, `year`
  - Flags: `is_zero_spend`, `is_high_spend`, `is_low_spend`
  - Demographics: `age`, `household_size`, `loyalty_member`, `is_high_value_customer`
- **Date Range:** 2025-09-06 to 2025-11-22 (12 weeks)
- **Key Fields:**
  - `customer_id`: Integer (1-200)
  - `store_id`: Integer (101 or 102)
  - `weekly_spend`: Float (can be 0.0)
  - `week_start_date`: String (YYYY-MM-DD format)

### Stock Data (`stock_availability.csv`)
- **Structure:** 360 records (2 stores × 15 SKUs × 12 weeks)
- **Columns:** 6 columns
  - `store_id`, `sku`, `week_start`, `on_hand_units`, `on_order_units`, `avg_weekly_demand`
- **Date Range:** 2025-09-06 to 2025-11-22 (12 weeks)
- **Key Fields:**
  - `store_id`: Integer (101 or 102)
  - `sku`: String (SKU-001 to SKU-015)
  - `week_start`: String (YYYY-MM-DD format)
  - `on_hand_units`: Integer (can be 0 for stockouts)

### Customer Data (`retail_customers.csv`)
- **Structure:** 200 records
- **Columns:** 4 columns
  - `customer_id`, `age`, `household_size`, `loyalty_member`
- **Note:** Already merged into sales data, no separate join needed

### Integration Points
- **Sales ↔ Stock:** Join on `store_id` and `week_start_date` = `week_start`
- **Sales ↔ Customer:** Already merged (customer demographics in sales data)

---

## 🏗️ Architecture Plan

### Directory Structure
```
agents/
  revenue_gap_orchestrator/
    __init__.py
    nodes.py                    # All node functions
    utilities/
      __init__.py
      data_loading.py          # Data loading utilities
      revenue_analysis.py      # Revenue analysis utilities
      gap_detection.py         # Gap detection utilities
      stockout_analysis.py      # Stockout impact utilities
      scoring.py                # Scoring utilities
      report_generation.py     # Report generation utilities
      llm_enhancement.py        # LLM utilities (Phase 8)
    orchestrator.py            # Main orchestrator workflow
```

### Node Structure (Linear Flow)
1. **goal_node** - Define analysis goal
2. **planning_node** - Create execution plan
3. **data_loading_node** - Load sales, stock, customer data
4. **revenue_analysis_node** - Calculate baselines, trends, predictions
5. **gap_detection_node** - Detect revenue gaps and churn risks
6. **stockout_analysis_node** - Analyze stockout impact (NEW - for stock data)
7. **scoring_node** - Score gaps by priority
8. **ranking_node** - Rank and select top N gaps
9. **report_generation_node** - Generate final report

---

## 📋 State Schema Verification

### ✅ State Schema Already Defined
The `PredictiveRevenueGapState` in `config.py` is comprehensive and includes:
- ✅ Input fields (customer_id, analysis_date, data_dir)
- ✅ Configuration fields (baseline_weeks, gap_thresholds, etc.)
- ✅ LLM config fields (already included - good!)
- ✅ Goal & Planning fields
- ✅ Data ingestion fields
- ✅ Revenue analysis fields
- ✅ Gap detection fields
- ✅ Opportunity detection fields
- ✅ Scoring & ranking fields
- ✅ Output fields
- ✅ Metadata fields

### Additional Fields Needed
Based on stock data integration, we may need:
- `stock_data: List[Dict[str, Any]]` - Loaded stock availability data
- `stockout_impact: List[Dict[str, Any]]` - Stockout impact analysis
- `merged_sales_stock: List[Dict[str, Any]]` - Joined sales + stock data

**Note:** These can be added to state as needed, or kept as intermediate variables.

---

## 🔄 Decision Rule Dependencies

### Revenue Analysis Rules
1. **Baseline Calculation:**
   - Input: Sales history, baseline_weeks config
   - Output: Average weekly spend for baseline period
   - Dependencies: None (first calculation)

2. **Trend Analysis:**
   - Input: Sales history, recent_weeks config
   - Output: Revenue trend (declining/stable/growing)
   - Dependencies: Baseline calculation

3. **Revenue Prediction:**
   - Input: Sales history, trend, prediction_horizon_weeks
   - Output: Predicted revenue
   - Dependencies: Trend analysis

### Gap Detection Rules
1. **Declining Revenue Gap:**
   - Input: Current revenue, baseline, declining_revenue_threshold
   - Output: Gap if decline > threshold
   - Dependencies: Revenue analysis

2. **Below Baseline Gap:**
   - Input: Current revenue, baseline, below_baseline_threshold
   - Output: Gap if below baseline by threshold
   - Dependencies: Revenue analysis

3. **Churn Risk:**
   - Input: Zero spend weeks, churn_risk_zero_weeks threshold
   - Output: Churn risk flag
   - Dependencies: Sales history

4. **Stockout Impact:**
   - Input: Stock data, sales data (joined)
   - Output: Stockout-related gaps
   - Dependencies: Data loading (both datasets)

### Scoring Rules
1. **Revenue Impact Score:**
   - Input: Gap amount, scoring_weights
   - Dependencies: Gap detection

2. **Churn Risk Score:**
   - Input: Churn risk factors, scoring_weights
   - Dependencies: Gap detection

3. **Customer Value Score:**
   - Input: Historical customer value, scoring_weights
   - Dependencies: Revenue analysis

4. **Recovery Probability Score:**
   - Input: Gap type, customer characteristics, scoring_weights
   - Dependencies: Gap detection, customer data

---

## 🧪 Testing Strategy

### Utilities First
- Test each utility function independently
- Use real data samples
- Verify edge cases (zero spend, missing data, etc.)

### Nodes Second
- Test nodes with mock state
- Verify state updates correctly
- Test error handling

### Integration Testing
- Test complete workflow end-to-end
- Verify state flows through all nodes
- Test with different customer_ids

---

## 📝 Next Steps

1. ✅ Phase 0: Planning (this document)
2. ⏳ Phase 1: Foundation (goal & planning nodes)
3. ⏳ Phase 2: Data Loading (utilities + node)
4. ⏳ Phase 3: Revenue Analysis (utilities + node)
5. ⏳ Phase 4: Gap Detection (utilities + node)
6. ⏳ Phase 5: Stockout Analysis (utilities + node) - NEW
7. ⏳ Phase 6: Scoring & Ranking (utilities + nodes)
8. ⏳ Phase 7: Report Generation (utilities + node)
9. ⏳ Phase 8: Wire up orchestrator
10. ⏳ Phase 9: LLM Enhancement (after MVP works)

---

**Status:** ✅ Planning Complete - Ready to Begin Phase 1



# Revenue Gap Orchestrator Nodes

In [ ]:
"""
Revenue Gap Orchestrator Nodes

Following MVP-first approach: rule-based nodes, no LLM dependencies.
"""

from typing import Dict, Any, List
from config import PredictiveRevenueGapState


def goal_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for revenue gap analysis.

    This is a simple rule-based goal definition that sets the framework.
    MVP: Fixed goal structure, no LLM needed.
    """
    customer_id = state.get("customer_id")
    analysis_date = state.get("analysis_date")

    # Build goal structure
    goal = {
        "objective": "Identify and prioritize revenue gaps for proactive intervention",
        "customer_id": customer_id,
        "analysis_date": analysis_date,
        "analysis_scope": "all_customers" if customer_id is None else "single_customer",
        "focus_areas": [
            "revenue_trend_analysis",
            "baseline_comparison",
            "churn_risk_detection",
            "stockout_impact_analysis",
            "recovery_opportunity_identification"
        ],
        "expected_outputs": [
            "revenue_gaps",
            "churn_risk_customers",
            "revenue_recovery_opportunities",
            "prioritized_gap_report"
        ]
    }

    errors = state.get("errors", [])

    return {
        "goal": goal,
        "errors": errors
    }


def planning_node(state: PredictiveRevenueGapState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    """
    goal = state.get("goal")
    errors = state.get("errors", [])

    if not goal:
        return {
            "errors": errors + ["planning_node: goal is required"]
        }

    # Create execution plan
    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load sales history, stock availability, and customer data",
            "dependencies": [],
            "outputs": [
                "sales_history",
                "stock_data",
                "all_customers",
                "sales_lookup"
            ]
        },
        {
            "step": 2,
            "name": "revenue_analysis",
            "description": "Calculate revenue baselines, trends, and predictions",
            "dependencies": ["data_loading"],
            "outputs": [
                "customer_revenue_baseline",
                "revenue_predictions"
            ]
        },
        {
            "step": 3,
            "name": "gap_detection",
            "description": "Detect revenue gaps, churn risks, and below-baseline performance",
            "dependencies": ["revenue_analysis"],
            "outputs": [
                "revenue_gaps",
                "churn_risk_customers"
            ]
        },
        {
            "step": 4,
            "name": "stockout_analysis",
            "description": "Analyze stockout impact on revenue gaps",
            "dependencies": ["data_loading", "gap_detection"],
            "outputs": [
                "stockout_impact"
            ]
        },
        {
            "step": 5,
            "name": "opportunity_detection",
            "description": "Identify recovery opportunities for detected gaps",
            "dependencies": ["gap_detection"],
            "outputs": [
                "revenue_recovery_opportunities"
            ]
        },
        {
            "step": 6,
            "name": "scoring",
            "description": "Score gaps by revenue impact, churn risk, customer value, and recovery probability",
            "dependencies": ["gap_detection", "opportunity_detection"],
            "outputs": [
                "scored_gaps"
            ]
        },
        {
            "step": 7,
            "name": "ranking",
            "description": "Rank gaps by priority and select top N",
            "dependencies": ["scoring"],
            "outputs": [
                "ranked_gaps",
                "top_priority_gaps",
                "gap_summary"
            ]
        },
        {
            "step": 8,
            "name": "report_generation",
            "description": "Generate final revenue gap report with recommendations",
            "dependencies": ["ranking"],
            "outputs": [
                "revenue_gap_report",
                "report_file_path"
            ]
        }
    ]

    return {
        "plan": plan,
        "errors": errors
    }



# Tests for Revenue Gap Orchestrator Foundation Nodes

In [ ]:
"""
Tests for Revenue Gap Orchestrator Foundation Nodes

Testing Phase 1: Goal and Planning nodes
"""

import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(__file__).parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

from agents.revenue_gap_orchestrator.nodes import goal_node, planning_node
from config import PredictiveRevenueGapState


def test_goal_node():
    """Test goal node with customer_id"""
    state: PredictiveRevenueGapState = {
        "customer_id": "1",
        "analysis_date": "2025-11-22",
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result
    assert result["goal"]["customer_id"] == "1"
    assert result["goal"]["objective"] == "Identify and prioritize revenue gaps for proactive intervention"
    assert "focus_areas" in result["goal"]
    assert len(result["goal"]["focus_areas"]) > 0
    assert "errors" in result
    print("✅ Goal node test passed")


def test_goal_node_all_customers():
    """Test goal node with None customer_id (all customers)"""
    state: PredictiveRevenueGapState = {
        "customer_id": None,
        "analysis_date": None,
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result
    assert result["goal"]["analysis_scope"] == "all_customers"
    assert result["goal"]["customer_id"] is None
    print("✅ Goal node (all customers) test passed")


def test_planning_node():
    """Test planning node with goal"""
    state: PredictiveRevenueGapState = {
        "goal": {
            "objective": "Identify and prioritize revenue gaps",
            "customer_id": "1",
            "analysis_scope": "single_customer"
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result
    assert len(result["plan"]) == 8  # Should have 8 steps
    assert result["plan"][0]["name"] == "data_loading"
    assert result["plan"][-1]["name"] == "report_generation"
    assert "errors" in result
    print("✅ Planning node test passed")


def test_planning_node_no_goal():
    """Test planning node error handling when goal is missing"""
    state: PredictiveRevenueGapState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result
    assert len(result["errors"]) > 0
    assert "goal is required" in result["errors"][0]
    print("✅ Planning node error handling test passed")


def test_goal_and_planning_flow():
    """Test goal and planning nodes together"""
    state: PredictiveRevenueGapState = {
        "customer_id": "1",
        "analysis_date": "2025-11-22",
        "errors": []
    }

    # Run goal node
    state = goal_node(state)
    assert "goal" in state

    # Run planning node
    state = planning_node(state)
    assert "plan" in state
    assert len(state["plan"]) == 8

    print("✅ Goal and planning flow test passed")


if __name__ == "__main__":
    print("Testing Revenue Gap Orchestrator Foundation Nodes...\n")

    test_goal_node()
    test_goal_node_all_customers()
    test_planning_node()
    test_planning_node_no_goal()
    test_goal_and_planning_flow()

    print("\n✅ All foundation node tests passed!")



In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_034_Predictive_Revenue_Gap_Orchestrator % cd python3 tests/test_revenue_gap_foundation.py
Testing Revenue Gap Orchestrator Foundation Nodes...

✅ Goal node test passed
✅ Goal node (all customers) test passed
✅ Planning node test passed
✅ Planning node error handling test passed
✅ Goal and planning flow test passed

✅ All foundation node tests passed!


Foundation tests passed. Proceeding to Phase 2: Data Loading — building utilities first, then the node.


Phase 2: Data Loading is complete. Created:

1. **Data loading utilities** (`utilities/data_loading.py`):
   - `load_sales_data()` - Load sales CSV
   - `load_stock_data()` - Load stock CSV
   - `load_customer_data()` - Load customer CSV
   - `build_sales_lookup()` - Create customer_id → sales records lookup
   - `filter_sales_by_customer()` - Filter by customer_id
   - `merge_sales_stock()` - Join sales and stock data
   - `convert_dataframe_to_dict_list()` - Convert DataFrames to state-compatible format

2. **Data loading node** (`nodes.py`):
   - Orchestrates all data loading
   - Handles single customer vs. all customers
   - Error handling

3. **Tests** (`test_data_loading_utilities.py`):
   - Tests for all utilities

## Progress summary

- Phase 0: Planning — Complete
- Phase 1: Foundation — Complete (goal & planning nodes tested)
- Phase 2: Data Loading — Complete (utilities + node ready for testing)

Next steps:
1. Test the data loading utilities and node
2. Continue with Phase 3: Revenue Analysis

